# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [ ]:
print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
path = "../starter_code/data.csv"
weather_df = pd.read_csv(path, index_col=0)
weather_df


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
max_humidity = weather_df['Humidity'].max()
max_humidity

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
locations = weather_df[["Lat", "Lng"]]
weight = weather_df["Humidity"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating = False, max_intensity = max_humidity,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
weather_df = weather_df.loc[(((weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0) & (weather_df["Country"] == "US")), :]

In [ ]:
hotel_df = weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.

params = {"radius": 50000, 
          "types": "hotel", 
          "keyword": "hotel",
          "key": g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_list = hotel_df[["City", "Country", "Hotel Name"]]
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
